In [ ]:
!pip install ipywidgets
!pip install control

In [2]:
import numpy as np
import control as ctrl
from scipy.signal import cont2discrete
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, Button, VBox, HBox, Output
from IPython.display import display, clear_output

In [3]:
np.set_printoptions(threshold=np.inf)

In [4]:
# PARÁMETROS DEFINITIVOS
Kp = 0.1
z_c = 1.0
tau = 0.3
a1 = 1.8
a0 = 0.9
Kq = 0.8
z_p = 1.2
kq = 0.5

Ki = Kp * z_c

In [5]:
# DeltaT
dt = FloatSlider(description='DeltaT', min=0, max=1, step=0.1, value=1)

# Tiempo de simulación
tiempoSimulacion = FloatSlider(description='Tiempo de Simulación', min=100, max=300, step=10, value=150.0)

# Valor de referencia
ref = FloatSlider(description='Referencia', min=-10.0, max=15.0, step=0.5, value=10.0)

# Segundo en el que se inserta la perturbación
pert_start = IntSlider(description='Pert. Inicio', min=0, max=99, step=1, value=70)

# Segundos que dura la perturbación
pert_dur = IntSlider(description='Duración', min=1, max=200, step=1, value=3)

# Amplitud de la perturbación
pert_amp = FloatSlider(description='Amplitud', min=0, max=50, step=1.0, value=20.0)

# Botón RUN
run_button = Button(description='Run', button_style='success')
output = Output()

In [6]:
u_max = 15   # grados de deflexión máxima
u_min = -15

activar_lim = True

# Gc = (Kps + Ki) / s = Kp + Ki/s
Gc = ctrl.tf([Kp, Ki], [1, 0])

# Ga = 1 / (tau*s + 1)
Ga = ctrl.tf([1], [tau, 1])

# Gp = (Kq*s + Kq* z_p) / (s^3 + a1 * s^2 + a0*s) = (Kq (s + z_p)) / (s(s^2 + a1 * s + a0))
Gp = ctrl.tf([Kq, Kq * z_p], [1, a1, a0, 0])
G1 = ctrl.feedback(Ga * Gp, kq)  # G1 = (Ga*Gp)/(1+Ga*Gp*kq)
H2 = 1

# G_total = (Gc*G1) / (1+Gc*G1)
G_global = ctrl.feedback(Gc * G1, H2)

# Polos
print("Polos del sistema total:", ctrl.poles(G_global))

Polos del sistema total: [-3.48117088+0.j         -1.01728038+0.j         -0.20764354+0.60694012j
 -0.20764354-0.60694012j -0.21959498+0.j        ]


In [7]:
# Creo un espacio de estado a partir de G1
G1_ss = ctrl.ss(G1)

# G1 se puede representar como un espacio de estado continuo, es decir, como dos ecuaciones diferenciales

# x'(t) = A x(t) + B u(t)
# y(t)  = C x(t) + D u(t)

# x es el vector de estado interno
# u es el vector de entrada de G1
# y es el vector de salida de G1

A = G1_ss.A
B = G1_ss.B
C = G1_ss.C
D = G1_ss.D

# Para simular en computadora trabajo en steps de tiempo de magnitud dt, por lo tanto, tengo que discretizar el espacio de estado continuo

# x[k+1] = Ad x[k] + Bd u[k]
# y[k]   = Cd x[k] + Dd u[k]

# Ad indica cómo evoluciona el estado del sistema entre un instante y el siguiente, considerando la dinámica de G1 y el tiempo de muestreo.
# Bd muestra cómo la entrada digitalizada u[k] afecta al estado siguiente.
# Cd indica cómo el estado x[k] se convierte en salida y[k].
# Dd indica si hay una conexión directa entre entrada y salida.

In [8]:
def simular(ref, pert_start, pert_dur, pert_amp, tiempoSimulacion, dt):
    t_vector = np.arange(0, tiempoSimulacion, dt)
    Ad, Bd, Cd, Dd, _ = cont2discrete((A, B, C, D), dt)

    salida = np.zeros_like(t_vector)
    errorAcum = 0
    x = np.zeros((Ad.shape[0],))
    # Hacemos el vector perturbación poniendo todo ceros menos en los segundos deseados
    pert = np.where((t_vector >= pert_start) & (t_vector < pert_start + pert_dur), pert_amp, 0)

    print("\n   Tiempo | Entrada |   Salida   ")
    print("-" * 35)

    # Simulación
    # Cada iteración representa un segundo o scan
    for i in range(1, len(t_vector)):
        # Perturbación en este instante t
        pert_act = pert[i]

        # Señal de error en este instante t
        señalDeError = ref - salida[i-1]

        # Como esta discretizado, en lugar de usar una integral se usa una sumatoria de errores acumulados en la variable integral
        señalDeControl = Kp * señalDeError + Ki * errorAcum + pert_act

        # errorAcum += señalDeError * dt

        if activar_lim:
            señalDeControlSat = np.clip(señalDeControl, u_min, u_max)
        else:
            señalDeControlSat = señalDeControl

        # @ hace el producto matricial

        # x[k+1] = Ad x[k] + Bd u[k]
        x = Ad @ x + Bd.flatten() * señalDeControlSat

        # y[k] = Cd x[k] + Dd u[k]
        y = Cd @ x + Dd.flatten() * señalDeControlSat
        salida[i] = y[0]  # y es un array con un solo valor

        if señalDeControl == señalDeControlSat:
            errorAcum += señalDeError * dt

        print(f"{t_vector[i]:>9.5f} | {ref:>7.2f} | {salida[i]:>10.4f}")

    # Vector de 100 posiciones con la entrada 10
    entrada = np.ones_like(t_vector) * ref

    plt.figure(figsize=(10, 5))
    plt.plot(t_vector, salida, label="Salida θ(t)")
    plt.plot(t_vector, entrada, '--', label="Entrada θc")
    plt.xlabel("Tiempo [s]")
    plt.ylabel("Ángulo de pitch")
    plt.title("Respuesta")
    plt.legend()

    plt.grid(axis='x', color='gray', linestyle='--', linewidth=0.5)

    step = 4 if t_vector[-1] <= 100 else 10
    plt.xticks(np.arange(0, t_vector[-1] + 1, step))


    plt.tight_layout()
    plt.show()

In [9]:
def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        simular(ref.value, pert_start.value, pert_dur.value, pert_amp.value, tiempoSimulacion.value, dt.value)

In [10]:
run_button.on_click(on_button_clicked)

In [11]:
display(VBox([
    HBox([ref, pert_start]),
    HBox([pert_dur, pert_amp]),
    HBox([tiempoSimulacion, dt]),
    run_button,
    output
]))